# Config

In [1]:
%pip install firebase-admin faker --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: C:\Users\49152\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import firebase_admin
from firebase_admin import credentials, firestore
from faker import Faker

In [3]:
cred = credentials.Certificate('serviceAccount.json')
firebase_admin.initialize_app(cred)

# Checkout the collections and documents

In [4]:
#get list of collections
db = firestore.client()
collections = [x.id for x in db.collections()]
print(collections)

[]


# Generate fake data and load into firestore

### Masterclasses Collection

In [5]:
#masterclassCollection = db.collection('Masterclasses')
fake = Faker()

list_of_levels = ["Beginner", "Intermediate", "Advanced"]
list_of_status = ["locked", "unlocked", "completed"]
list_class_room = [fake.city() for i in range(30)]

In [6]:


for i in range(100):
    data = {
        "art": "Masterclass",
        "author": fake.name(),
        "classroom" : [list_class_room[fake.random_int(0, 29)] for i in range(fake.random_int(1, 5))],
        "courseLessons" : [fake.random_int(1, 5) for i in range(fake.random_int(1, 5))],
        "courseOverview": {"AboutAuthor" : fake.text(50) + "...", "AboutCourse" : fake.text(50) + "..."},
        "description" : fake.text(max_nb_chars=50) + "...",
        "duration" : fake.random_int(1, 100),
        "filterTags": [fake.word() for i in range(fake.random_int(1, 5))],
        "id": i+2,
        "lessons": fake.random_int(1, 100),
        "level": list_of_levels[fake.random_int(0, 2)],
        "points": fake.random_int(1, 100),
        "progress": 0,
        "status": list_of_status[fake.random_int(0, 2)],
        "tags" : [fake.word() for i in range(fake.random_int(1, 5))],
        "thumbnail" : fake.image_url(["facebook"]),
        "title" : fake.text(max_nb_chars=20),
        "videoUrl" : fake.url(["video"]),
        "enrollment_count": fake.random_int(0, 100)
    }
    #masterclassCollection.add(data)
    print(data)


### Users Collection

In [7]:
import random
#usersCollection = db.collection('Users')
fake = Faker()
list_of_account_Types = ["admin", "master", "superadmin", "instructor"]
list_of_Education = ["Schule", "Abitur", "Realschulabschluss", "Hochschulabschluss", "Bachelor", "Master", "Promotion", "Ausbildung"]
course_enrolled = fake.random_int(0, 40)
for i in range(100): 
    data = {
        "id": i + 2,
        "accType": "Customer" if random.random() < 0.95 else list_of_account_Types[fake.random_int(0, 3)],
        "advisor": fake.name(),
        "advisorACID": "advisor reference" + str(i),
        "city": fake.city(),
        "email": fake.email(),
        "firstName": fake.first_name(),
        "joined": fake.date(),
        "lifetime" : fake.random_int(0, 5),
        "postalCode": fake.postcode(),
        "refACID": "reference" + str(i),
        "role": fake.job(),
        "surname": fake.last_name(),
        "Gender": list(["Male", "Female"])[fake.random_int(0, 1)],
        "Income": fake.random_int(10000, 80000),
        "Birthdate": fake.date_of_birth().strftime('%Y/%m/%d'),
        "Education": list_of_Education[fake.random_int(0, len(list_of_Education) - 1)],
        "Employment_status": ["unemployed", "employed"][fake.random_int(0, 1)],
        "Watched": [ {"Masterclass" : fake.random_int(2, 101), "progress": str(fake.random_int(0, 500)) + " min"} for i in range(fake.random_int(0, 5))],
        "Interest": [{"classroom": list_class_room[fake.random_int(0, 29)],"score": fake.random_int(0, 100) } for i in range(0,29)],
        "Total_Watchtime" : str(fake.random_int(0, 100000)) + " min",
        "Avg_session_duration": str(fake.random_int(0, 200)) + " min",
        "Courses_enrolled" : course_enrolled,
        "Courses_completed" : fake.random_int(0, course_enrolled),
        "course_saved": fake.random_int(0, course_enrolled)
    }
    #usersCollection.add(data)
    print(data)

### Coach Collection

In [8]:
#coachCollection = db.collection('Coaches')
fake = Faker()

for i in range(100): 
    data = {
        "about": fake.text(50),
        "blogs": [fake.text(50) + "..." for i in range(fake.random_int(1, 5))],
        "books": [fake.word() for i in range(fake.random_int(1, 5))],
        "facebook": fake.url(),
        "id": i+2,
        "image": [fake.name()+ ".jpg" for i in range(fake.random_int(1, 5))],
        "instagram": fake.url(["instagram"]),
        "linkedin": fake.url(["linkedin"]),
        "link": fake.url(),
        "masterclass": [{"name": fake.name()} for i in range(fake.random_int(1, 5))],
        "podcasts" : [{"position": fake.job(), "website": fake.url(), "xing": fake.url()} for i in range(fake.random_int(1, 5))],
        "Name": fake.name(),
        "position": fake.job()
    }
    #coachCollection.add(data)
    print(data)

In [9]:
#eventCollection = db.collection('Events')
fake = Faker()
from datetime import date

Event_Types = ["enroll_course", "save_course", "unsave_course", "complete_course", "complete_lesson", "login", "logout", "Berater_kontaktieren", "Berater_wechseln"]
for i in range(200): 
    data = {
        "Event-ID": i + 2,
        "User-Id": fake.random_int(2, 101),
        "Timestamp": fake.date(),
        "Event-Type" : Event_Types[fake.random_int(0, len(Event_Types) - 1)],
        "Event-Specific-Information" : fake.text()
    }
    #eventCollection.add(data)
    print(data)


In [10]:
# nicht nutzen wegen täglichem Limit von Operationen
def delete_collection(coll_ref, batch_size):
    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0

    for doc in docs:
        print(f"Deleting doc {doc.id} => {doc.get().to_dict()}")
        doc.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)
#db = firestore.client()
#collections = [x for x in db.collections()]
#for collection in collections:
    #delete_collection(collection, 0)